# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Оглавление<a name="Оглавление"></a>

- 1. [Ознакомление с данными](#Ознакомление)
- 2. [Разбиение данных на выборки](#Разбиение)
- 3. [Исследование моделей](#Исследование)
  - 3.1. [DecisionTreeClassifier](#DecisionTreeClassifier)
  - 3.2. [RandomForestClassifier](#RandomForestClassifier)
  - 3.3. [LogisticRegression](#LogisticRegression)
  - 3.4. [Вывод по моделям](#Вывод_по)
- 4. [Проверка лучшей модели на тестовой выборке](#Проверка)
  - 4.1. [Проверка модели на адекватность](#адекватность)
- 5. [Вывод](#Вывод)

## 1. Ознакомление с данными<a name="Ознакомление"></a>

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from joblib import dump
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


[К оглавлению](#Оглавление)

## 2. Разбиение данных на выборки<a name="Разбиение"></a>

Мы умеем делить только на две части , поэтому сначала отделим 40% от всего датасета, а потом эти 40 % поделим пополам между тестовой и валидационной выборкой. Таким образом в них будет по 20 % от всего датасета.

In [4]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)

In [5]:
df_train.shape

(1928, 5)

In [6]:
df_valid.shape

(643, 5)

In [7]:
df_test.shape

(643, 5)

Подготовим поделенные данные ( поделим на прикнаки и целевой признак)

In [8]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

[К оглавлению](#Оглавление)

## 3. Исследование моделей<a name="Исследование"></a>

У нас целевой прризнак это столбец "is_ultra". В нем всего два вариатна значений 1 или 0, следовательно этот признак явяется категориальным. Будем решать задачу классификации, используя модели машинного обучения: Решающее дерево, Случайный лес, Логистическая регрессия.

### 3.1. DecisionTreeClassifier<a name="DecisionTreeClassifier"></a>

In [9]:
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth = depth)
    model.fit(features_train,target_train) 
    predictions_valid = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions_valid) 
    print('max_depth =', depth, ':', result)

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944
max_depth = 6 : 0.7838258164852255
max_depth = 7 : 0.7822706065318819
max_depth = 8 : 0.7791601866251944
max_depth = 9 : 0.7822706065318819
max_depth = 10 : 0.7744945567651633


Лучший результат при параметре глубины: max_depth = 3 : 0.7853810264385692.

[К оглавлению](#Оглавление)

### 3.2. RandomForestClassifier<a name="RandomForestClassifier"></a>

In [10]:
best_model = None
best_result = 0
for depth in range(2, 11):
    for est in range(1, 21):
        model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        model.fit(features_train,target_train) 
        result = model.score(features_valid, target_valid) 
        if result > best_result:
            best_model = model 
            best_result = result 

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print(best_model)

Accuracy наилучшей модели на валидационной выборке: 0.8040435458786936
RandomForestClassifier(max_depth=6, n_estimators=12, random_state=12345)


[К оглавлению](#Оглавление)

### 3.3. LogisticRegression<a name="LogisticRegression"></a>

In [11]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train,target_train)

predictions = model.predict(features_valid)

accuracy_score(target_valid, predictions_valid)

0.7744945567651633

### 3.4. Вывод по моделям<a name="Вывод_по"></a>
Сравнение моделей и их точность:
1) Решающее дерево 0.7853810264385692

2) Случайный лес 0.8040435458786936

3) Логистическая регрессия 0.7744945567651633

В итоге мы выбираем Случайный лес

[К оглавлению](#Оглавление)

## 4. Проверка лучшей модели на тестовой выборке<a name="Проверка"></a>

In [12]:
model = best_model
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
print('Аccuracy score:', accuracy_score(target_test, predictions_test))

Аccuracy score: 0.7947122861586314


## 4.1. Проверка модели на адекватность<a name="адекватность"></a>

In [13]:
constant = pd.Series([0]*len(target_test))
constant.value_counts()

0    643
dtype: int64

In [14]:
print('Аccuracy score константой модели:', accuracy_score(target_valid, constant))
print('Аccuracy score лучшей модели:', accuracy_score(target_test, predictions_test))

Аccuracy score константой модели: 0.7060653188180405
Аccuracy score лучшей модели: 0.7947122861586314


[К оглавлению](#Оглавление)

## 5. Вывод<a name="Вывод"></a>

Мы рассмотрели три модели: "Решающее дерево", "Случайный лес" и "Логистическая регрессия". В каждой модели, изменяя ее гиперпараметры, мы нашли нашли лучший вариант с наибольшим количеством правильных ответов. Путем простого сравнения лучших моделей по доле правильных ответов определили, что для нашей задачи подходит модель "Случайного леса" с гиперпараметрами глубина дерева 6 и количество деревьев 12.

Сравнив нашу модель со случайными предсказаниями и оценив ее качество на тестовом наборе данных (результат составил 79% правильных ответов) мы подтвердили, что наша модель подходит для задачи заказчика. Таким образом можно использовать данную модель для предложения клиентам заказчика нового тарифа ("Смарт" или "Ультра").